# Projekt MSP1
Cílem tohoto projektu je se seznámit s programovými nástroji využívaných ve statistice a osvojit si základní procedury. Projekt není primárně zaměřen na efektivitu využívání programového vybavení (i když úplně nevhodné konstrukce mohou mít vliv na hodnocení), ale nejvíce nás zajímají vaše statistické závěry a způsob vyhodnocení. Dbejte také na to, že každý graf musí splňovat nějaké podmínky - přehlednost, čitelnost, popisky.

V projektu budete analyzovat časy běhu šesti různých konfigurací algoritmů. Ke každé konfiguraci vzniklo celkem 200 nezávislých běhů, jejichž logy máte k dispozici v souboru [logfiles.zip](logfiles.zip).

Pokud nemáte rozchozené prostředí pro pro spouštění Jupyter notebooku, můžete využití službu [Google Colab](https://colab.google/). Jakákoliv spolupráce, sdílení řešení a podobně je zakázána!

S případnými dotazy se obracejte na Vojtěcha Mrázka (mrazek@fit.vutbr.cz).

__Odevzdání:__ tento soubor (není potřeba aby obsahoval výstupy skriptů) do neděle 22. 10. 2023 v IS VUT. Kontrola bude probíhat na Pythonu 3.10.12; neočekává se však to, že byste používali nějaké speciality a nekompatibilní knihovny. V případě nesouladu verzí a podobných problémů budete mít možnost reklamace a prokázání správnosti funkce. Bez vyplnění vašich komentářů a závěrů do označených buněk nebude projekt hodnocen!

__Upozornění:__ nepřidávejte do notebooku další buňky, odpovídejte tam, kam se ptáme (textové komentáře do Markdown buněk)

__Tip:__ před odevzdáním resetujte celý notebook a zkuste jej spustit od začátku. Zamezíte tak chybám krokování a editací, kdy výsledek z buňky na konci použijete na začátku.

__OTÁZKA K DOPLNĚNÍ:__

Adam Zvara (xzvara01)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from zipfile import ZipFile

## Načtení dat do DataFrame
Ze souboru `logfiles.zip` umístěném ve stejném adresáři načtěte data a vytvořte Pandas DataFrame.

Z logu vás budou nejvíce zajímat řádky
```
Configuration: config6
Run: 191
Time of run: 53.298725254089774
```

Můžete využít následující kostru - je vhodné pracovat přímo se ZIP souborem. Jedinou nevýhodou je to, že vám bude vracet _byte_ objekt, který musíte přes funkci `decode` zpracovat.

In [ ]:
mappings = {"Configuration": "conf", "Run": "run", "Time of run": "time"}

def load_logfile(f) -> dict:
    """Load a logfile from a file-like object and return a dict with the data."""
    data = {
        "conf": None,
        "run": None,
        "time": np.nan
    }

    for line in f:
        line = line.decode("utf-8")
        line = line.split(":")
        if line[0] in mappings:
            data[mappings[line[0]]] = line[1].strip()
        
    return data

data = []
with ZipFile("logfiles.zip") as zf:
    for filename in zf.namelist():
        with zf.open(filename, "r") as f:
            data.append(load_logfile(f))
df = pd.DataFrame(data)
df

## Analýza a čištění dat
Vhodným způsobem pro všechny konfigurace analyzujte časy běhů a pokud tam jsou, identifikujte hodnoty, které jsou chybné. 

In [ ]:
df["time"] = df["time"].astype(float)
ax = df.boxplot(column="time", by="conf")
ax.set_yscale("log")

__OTÁZKA K DOPLNĚNÍ:__

_Objevily se nějaké chybné hodnoty? Proč tam jsou s ohledem na to, že se jedná o běhy algoritmů?_

__ODPOVEĎ:__

Každý zo skúmaných behov obsahuje okrajové hodnoty, pri ktorých je čas > 1000 alebo naopak je čas hodnota blízka nule. Keďže pozorujeme behy algoritmov, môžeme predpokladať, že sa jedná o "edge-cases" spôsobené nejakými chybami - napr. násilné ukončenie príliš dlhého behu operačným systémom (príp. iným prostriedkom) alebo priskoré ukončenie algoritmu zapríčené rôznymi chybami.

Vyčistěte dataframe `df` tak, aby tam tyto hodnoty nebyly a ukažte znovu analýzu toho, že čištění dat bylo úspěšné. Odtud dále pracujte s vyčištěným datasetem.

In [ ]:
df2 = df[(df["time"] > 1) & (df["time"] < 1000)]
df2.boxplot(column="time", by="conf")

## Deskriptivní popis hodnot
Vypište pro jednotlivé konfigurace základní deskriptivní parametry času pro jednotlivé konfigurace.  

__TIP__ pokud výsledky uložíte jako Pandas DataFrame, zobrazí se v tabulce.

In [ ]:
df2.groupby("conf").describe()

__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte, co všechno můžeme z parametrů vyčíst._

__ODPOVEĎ:__

- **počet hodnôt** - počet hodnôt sa pri jednotlivých konfiguráciách príliš nelíši, takže nad nimi môžeme realizovať štatistickú analýzu 
- **priemer** - z hľadiska aritmetického priemeru môžeme považovať konfigurácie _config{1,4}_ za najrýchlejšie, zatiaľ čo _config{2,5,6}_ za najpomalšie
- **štandardná odchylka** - konfigurácia _config1_ má najmenšiu STD, teda hodnoty sú málo "rozprestrené" a od tejto konfigurácie môžeme očakávať najkonzistnejšie výsledky {Opak platí pre _config4_)
- **minimum a maximum** - minimálna hodnota nám udáva vhodných kandidátov na bližie preskúmanie (_config{1,4}_), a zároveň
- **{25, 50, 75} percentil** - určujú hranice, pod ktorými sa nachádza *n%* všetkých hodnôt

## Vizualizace
Vizualizujte časy běhů algoritmů v jednom kompaktním grafu tak, aby byl zřejmý i rozptyl hodnot. Zvolte vhodný graf, který pak níže komentujte.

In [ ]:
ax = sns.boxplot(x="conf", y="time", data=df2, hue="conf")
ax.grid(True)

__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte  výsledky z tabulky._

__ODPOVEĎ:__

Na vizualizáciu časov behu algoritmov môžeme použiť boxplot z knižnice `seaborn`, kde na x-ovej ose zobrazujeme konfigurácie jednotlivých behov a na y-ose čas behu. Boxplot je vhodný na zobrazenie deskriptívnych popis hodnôt roznych behov algoritmov, pretože v grafe priamo vidíme porovnanie rôznych konfigurácií, spolu s mediánom (prostredná čiara v boxe), prvý (25%) a tretí (75%) kvartil - hranice "boxu" alebo "outliers" (okrajové hodnoty mimo boxplotu). Na prvý pohľad môžeme vidieť, že konfigurácie 1 a 4 sú vhodný kandidáti na skúmanie (hľadáme najnižšie hodnoty -> sú "nižšie" na y-ose oproti ostatným konfiguráciám)


## Určení efektivity konfigurací algoritmů
Nás ale zajímá, jaká konfigurace je nejrychlejší. Z výše vykresleného grafu můžeme vyloučit některé konfigurace. Existuje tam však minimálně jedna dvojice, u které nedokážeme jednoznačně určit, která je lepší - pokud nebudeme porovnávat pouze extrémní hodnoty, které mohou být dané náhodou, ale celkově. Proto proveďte vhodný test významnosti - v následující části diskutujte zejména rozložení dat (i s odkazem na předchozí buňky, variabilitu vs polohu a podobně). Je nutné každý logický krok a výběry statistických funkcí komentovat. Můžete i přidat další buňky.

Vužijte vhodnou funkci z knihovny `scipy.stats` a funkci poté __implementujte sami__ na základě základních matematických funkcí knihovny `numpy` případně i funkcí pro výpočet studentova rozložení v [scipy.stats](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html). Při vlastní implementaci není nutné se primárně soustředit na efektivitu výpočtu (není potřeba využít všechny funkce numpy, můžete použít normální cykly a podobně - v hodnocení však bude zahrnuta přehlednost a neměly by se objevit jasné chyby, jako je zvýšení třídy složitosti a podobně).

__OTÁZKA K DOPLNĚNÍ:__

_Jaká data budete zkoumat? Jaké mají rozložení a parametry (např. varianci) a jaký test použijete? Jaká je nulová hypotéza? Jak se liší variabilita a poloha vybraných konfigurací?_

__ODPOVEĎ:__

Z doposiaľ skúmaných konfigurácií sa môžeme venovať najmä dvojici _{conf1, conf4}_, pretože pri nich môžeme vidieť v porovnaní s ostatnými konfiguráciami najlepšie výsledky z hľadiska behu programu. Keďže je vzorka dát pomerne veľká, môžeme aplikovať CLV a pracovať s nimi ako s normálnymi rozdeleniami. Na identifikáciu lepšieho algoritmu použijeme dvojvýberový (porovnávame hodnoty 2 rozdelení) nepárový (hodnoty sú nezávislé) T-test na porovnanie zhody stredných hodnôt rozdelení. Nulová hypotéza pri tomto teste je H<sub>0</sub>: &mu;<sub>conf1</sub> = &mu;<sub>conf4</sub> a pri alternatívnej si zvolím variant jednostranného testu H<sub>1</sub>: &mu;<sub>conf1</sub> < &mu;<sub>conf4</sub>, pri ktorom testujem, že konfigurácia _conf1_ je rýchlejšia ako konfigurácia _conf4_.

Na realizáciu T-testu potrebujeme (všeobecne) vedieť, či majú dané rozdelenia rovnakú variaciu (i keď pri veľkom množstve dát je možné túto podmienku nesplniť), čo ale vidíme už na prvý pohľad z tabuľky deskriptívnych hodnôt, kde<br>
&sigma;<sup>2</sup><sub>conf1</sub> = 9.386788<sup>2</sup> = 88.11179<br>
&sigma;<sup>2</sup><sub>conf4</sub> = 30.656816<sup>2</sup> = 939.84034<br>

Knihovna scipy poskytuje implementáciou nepárového T-testu funkciou `ttest_ind`, kedy môžeme parametrom zadať nerovnosť rozptylov a zároveň si môžeme vybrať alternatívnu hypotézu.

In [ ]:
x = df2[df2["conf"] == "config1"]["time"]
y = df2[df2["conf"] == "config4"]["time"]

stats.ttest_ind(x, y, equal_var=False, alternative="less")

__OTÁZKA K DOPLNĚNÍ:__

_Jaký je závěr statistického testu?_

Štatistický test realizovaný knihovnou funkciou `ttest_ind` sa zhoduje s ručne implementovaným testom a teda výsledok oboch testov je, že stredná hodnota konfigurácie _conf1_ je **významne štasticky menšia** ako stredná hodnota konfigurácie _conf4_. Pri oboch testoch vidíme zhodu v získanej t-hodnote, pričom pri prvom teste môžeme porovnávať že výsledná p-hodnota je menšia ako štandardná hladina významnosti 0.5 - teda nulovú hypotézu zamietame. V ručne implementovanom teste pracujeme čisto s t-hodnotou, ktorá je menšia ako získaná (záporná - pretože sme na ľavo od osy x) tabuľková hodnota pre t-test s danou hladinou významnosti (0.5 pre jednostranný test) a stupňom voľnosti v závislosti od dát. 

In [ ]:
t = (x.mean() - y.mean()) / np.sqrt(y.var() / y.count() + x.var() / x.count())
t_critical = - stats.t.ppf(0.95, x.count() + y.count() - 2)
if (t < t_critical):
    print("Zamietame nulovu hypotezu, rozdiel je signifikantny v prospech config1")
else:
    print("Nepodarilo sa zamietnut nulovu hypotezu")